## EDA-4 Статистические тесты. Домашнее задание

### Задача: 
Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов `petersburg` и `magadan` в разделе `ANOVA`-тест, проверьте данные на нормальность и на наличие корреляции.

### Метрики:

* Данные объединены в DataFrame
* Проведён тест на корреляцию
* Обоснован выбор теста на корреляцию
* Сделан правильный вывод по гипотезе
* Проведён тест на сравнение выборок
* Обоснован выбор теста на сравнение
* Сделан правильный вывод по гипотезе

In [29]:
import pandas as pd

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

# Запишем данные в DataFrame
df_1 = pd.DataFrame(petersburg, columns=['petersburg'])
df_2 = pd.DataFrame(magadan, columns=['magadan'])

# Объединим DF по столбцам
data = pd.concat([df_1, df_2],axis=1)

# Жалко удалять строчку с пропуском (это 15% данных), заполним пропуск средним значением
data = data.fillna(round(data['petersburg'].mean(),4))
data

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1034,0.0689


### Проверка данных на нормальность

In [32]:
# Выдвенем гипотезы
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

# Установим уровень значимости
alpha = 0.05

Используя тест Шапиро-Уилка определим насколько вероятно, что данные получены из распределения Гаусса (распределены нормально)

> Тест Шапиро-Уилка отлично себя показывает на небольших выборках, которой без сомнения является наш DataFrame

In [39]:
from scipy.stats import shapiro

# Вычисляем статистическую значимость
_, p = shapiro(data)

# Сравниваем полученную статистическую значимость с заданным уровнем значимости (5%)
if p > alpha:
	print(H0)
else:
	print(Ha)

Данные распределены нормально


### Тест на корреляцию

Для определения взаимосвязи между исследуемыми переменным используем **коэффициент кореляции Пирсона**, потому что:

* Данные распределены **нормально**
* Данные представлены **непрерывными** признаками
* В данных отсутствую **выбросы и аномалии**

Чтобы определить, является ли коэффициент корреляции между двумя переменными статистически значимым, выполним корреляционный тест с использованием функции `pearsonr` из библиотеки `SciPy`.

In [71]:
from scipy.stats import pearsonr

# Определим коэффициент корреляции
k = pearsonr(data['petersburg'], data['magadan'])[0]

print('Вывод:')

# Определим силу связи (коэффициент возьмем по модулю)
if abs(k) <= 0.3:
    print(f"Коэффициент корреляции равен {round(k,2)}. Связь очень слабая или отсутствует.")
else:
    print(f"Коэффициент корреляции равен {round(k,2)}. Связь есть, необходим дальнейший анализ")

Вывод:
Коэффициент корреляции равен 0.24. Связь очень слабая или отсутствует.


Теперь для ответа на вопрос "Какова разница в среднем размере мидии в зависимости от города-производителя?" следует воспользоваться **независимым t-тестом** так как:
* Данные распределены **нормально**
* Тип зависимой переменной - **количественная**
* Сравниваются **две** группы
* Объектов в выборке **< 30**

In [53]:
from scipy.stats import ttest_ind

# Выдвигаем гипотезы
H0 = 'Нет значимой разницы между размером мидий в зависимости от города-производителя.'
Ha = 'Есть значимая разница между размером мидий в зависимости от города-производителя.'

# Задаем уровень значимости
alpha = 0.05

# проводим t-тестирование
test_results = ttest_ind(data['petersburg'], data['magadan'], equal_var=True)

p = round(test_results[1],3)

# Сравниваем результат с заданным уровнем
if p>alpha:
    print(f"{p} > {alpha}.\n Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. \nМы отвергаем нулевую гипотезу. \n{Ha}")



0.003 <= 0.05. 
Мы отвергаем нулевую гипотезу. 
Есть значимая разница между размером мидий в зависимости от города-производителя.
